In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

C:\Users\Okita\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [24]:
ds = pd.read_csv('zombie-health-cases500.csv')
num_symp = len(ds.columns) - 1
ds = pd.read_csv('zombie-health-cases500.nsv')

ds = ds.drop('Unnamed: 0', axis=1)
symp = []
for i in ds.columns[:num_symp]:
    symp.append(i)
print(symp)
ds.head()

['paralysis', 'yellow_tongue', 'trembiling_finger', 'member_loss', 'chest_pain', 'severe_anger']


,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger,bacterial_infection,fight,viral_infection,zulombriga
0,0,0,0,0,1,0,0,0,1,0
1,0,1,1,0,0,0,0,0,0,1
2,0,0,0,1,1,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,1
4,0,1,1,0,0,0,0,0,0,1


In [4]:
ds = pd.read_csv('zombie-health-new-cases500.csv')
def encoder(ds):
    enc = LabelEncoder()
    for i in range(len(ds.columns) -1):
        ds.iloc[:, i] = enc.fit_transform(ds.iloc[:, i])
    return ds

def hot_encoder(ds):
    num_diagnostic = len(ds.columns) -1
    one_hot = pd.get_dummies(ds.iloc[:, num_diagnostic])
    ds = ds.drop(ds.columns[num_diagnostic], axis=1)
    ds = ds.join(one_hot)
    return ds

def process_dataset(dataset):
    ds = encoder(dataset)
    ds = hot_encoder(ds)
    return ds
ds = process_dataset(ds)
ds.head()


,Paralisia,Língua Amarela,Dedo Tremendo,Perda de Membro,Dor no Peito,Raiva Severa,Olho vermelho,Pele azul,Briga,Gripe Aviaria,Infecção Bacteriana,Infecção Viral,T-Virus,Zulombriga
0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,1,1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,1,1,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,1,0,0,0,0,1,0


In [50]:
ds = pd.read_csv('zombie-health-cases500.nsv')
ds = ds.drop(ds.columns[0], axis=1)
ds.head()

,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger,bacterial_infection,fight,viral_infection,zulombriga
0,0,0,0,0,1,0,0,0,1,0
1,0,1,1,0,0,0,0,0,0,1
2,0,0,0,1,1,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,1
4,0,1,1,0,0,0,0,0,0,1


In [51]:
ds_certo = ds
corr = ds.corr()
corr = corr.iloc[len(ds.columns) - num_symp +2:, :num_symp]
corr

,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger
bacterial_infection,0.580886,0.713886,-0.305610,0.007113,-0.466024,-0.316634
fight,-0.249787,-0.431648,-0.292907,0.295551,0.147556,0.940270
viral_infection,-0.104984,-0.457392,-0.310377,0.020804,0.728431,-0.321573
zulombriga,-0.240300,0.176698,0.943360,-0.330892,-0.429691,-0.291948


In [55]:
def achar_maior(corr):
    max = 0
    for i in range(len(corr)):
        for j in range(len(corr.columns)):
           if corr.iloc[i,j] > max:
                max = corr.iloc[i,j]
                sintoma = j
                doenca = i
    corr.iloc[doenca, sintoma] = 0
    return [sintoma, doenca]

corr = ds.corr()
corr = corr.iloc[len(ds.columns) - num_symp +2:, :num_symp]
ask_order = [['Symp', 'Disease']]
vetor = achar_maior(corr)
print(diseases)
print(vetor)
vetor[0] = symp[vetor[0]]
vetor[1] = diseases[vetor[1]]
print(vetor)
print(diseases)
ask_order.append(vetor)
corr

['bacterial_infection', 'fight', 'viral_infection', 'zulombriga']
[2, 3]
['trembiling_finger', 'zulombriga']
['bacterial_infection', 'fight', 'viral_infection', 'zulombriga']


,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger
bacterial_infection,0.580886,0.713886,-0.305610,0.007113,-0.466024,-0.316634
fight,-0.249787,-0.431648,-0.292907,0.295551,0.147556,0.940270
viral_infection,-0.104984,-0.457392,-0.310377,0.020804,0.728431,-0.321573
zulombriga,-0.240300,0.176698,0.000000,-0.330892,-0.429691,-0.291948


In [8]:
keys = ask_order[0]
data = [dict(zip(keys, values)) for values in ask_order[1:]]
data

[{'Symp': 6, 'Disease': 1},
 {'Symp': 4, 'Disease': 3},
 {'Symp': 2, 'Disease': 5},
 {'Symp': 1, 'Disease': 2},
 {'Symp': 5, 'Disease': 0},
 {'Symp': 0, 'Disease': 2},
 {'Symp': 5, 'Disease': 4},
 {'Symp': 7, 'Disease': 4}]

In [9]:
print(symp)
x = ds[symp]
y = 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=0)

SyntaxError: invalid syntax (<ipython-input-9-a2f54507a1a1>, line 3)

In [54]:
    dataset = 'zombie-health-cases500.csv'
    ds = pd.read_csv(dataset)

    symp = []
    for i in ds.columns[:-1]:
        symp.append(i)

    dataset = dataset.replace('.csv', '.nsv')
    ds = pd.read_csv(dataset)
    ds = ds.drop('Unnamed: 0', axis=1)
    diseases = []
    for i in ds.columns:
        if i not in symp:
            diseases.append(i)
            
    diseases

['bacterial_infection', 'fight', 'viral_infection', 'zulombriga']

In [15]:
import numpy as np
def gerar_modelo(symp, disease, ds):
    x = ds[symp]
    y = ds[disease]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=0)
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    return model

patient = [[0,1,0,0,0,0,1,0]]

modelos = []
for i in diseases:
    modelos.append(gerar_modelo(symp, i, ds))

probs = []
for model in range(len(modelos)):
    probs.append([model, modelos[model].predict(patient).tolist()[0]])
    
probs

[[0, 0.0], [1, 1.0], [2, 0.3258741258741259], [3, 0.0], [4, 0.0], [5, 0.0]]

In [21]:
def get_highest(prob_array):
    i = 0
    n = len(prob_array)
    maiores = [-1, -2] #maior é o -1
    ids = [-1, -1]
    while i < n:
        if prob_array[i][1] > maiores[0]:
            maiores[1] = maiores[0]
            ids[1] = ids[0]
            maiores[0] = prob_array[i][1]
            ids[0] = prob_array[i][0]
        elif prob_array[i][1] > maiores[1]:
            maiores[1] = prob_array[i][1]
            ids[1] = prob_array[i][0]
        i+=1;
    return ids

get_highest(probs)

[1, 2]